In [1]:
using Pkg
Pkg.activate(".")

# add projects
Pkg.add(url="https://github.com/jcsyme/IterativeHeaps.jl")
Pkg.add(url="https://github.com/jcsyme/GraphDistanceAlgorithms.jl")
Pkg.add(url="https://github.com/jcsyme/GraphFragments.jl")
Pkg.add(url="https://github.com/jcsyme/DiscreteGraphAlgorithms.jl")
Pkg.add("BenchmarkTools")


  Activating new project at `~/git/GraphDistanceAlgorithms.jl`


# Load Benchmarking and `Distributed`
- `Distributed` package is required to take advantage of parallelization
    - The `@everywhere` macro will be used below to ensure that parallelization can function using methods in each of the pacakges we need
    - Use `Sys.CPU_THREADS` to see how many cores (physical and virtual) you have at your disposal
- `BenchmarkTools` is used to compare performance

In [12]:
Sys.CPU_THREADS

10

In [13]:
using BenchmarkTools 
using Distributed

# add processes? the number of processes should be based on your system. You can use 
(nprocs() == 1) && addprocs(Sys.CPU_THREADS)

# load using everywhere macro to make sure cores can recognize objects
@everywhere using Graphs
@everywhere using GraphDistanceAlgorithms
@everywhere using GraphFragments

In [30]:
?GraphFragments.fragmentation

Calculate the fragmentation of a graph (KPP-Negative)

# Constructs

```
fragmentation(
    graph::Union{AbstractGraph, Nothing}, 
    dict_arrays::Union{Dict{Symbol, Vector}, Dict{Symbol, DArray}, Nothing} = nothing;
    D_invs::Union{Matrix{Int64}, Matrix{Float64}, Nothing} = nothing,
    distance_algorithm::Symbol = :auto,
    parallel_approach::Symbol = :auto,
    use_distance_weighting::Bool = true,
    kwargs...
)
```

```
fragmentation(
    adj::Union{SparseMatrixCSC{Float64, Int64}, Matrix{Float64}};;
    kwargs...
)
```

## Function Arguments

  * `graph`: graph on which to calculate fragmentation
  * `dict_arrays`: optional dictionary mapping relevant algorithm keys to arrays–   DArrays, SharedArrays (not recommended unless size is very large), or    Vectors–storing intermediate calculations.

      * Only effective if fixing `distance_algorithm` to align with the arrays   that are passed
      * See ?GraphDistanceAlgorithms.spawn_arrays for more information on the    inputs required.

## Keyword Arguments

  * `D_invs`: optional matrix (with 0 diagonal) of inverse distances. Passing    this optional argument can speed up calculations considerably.

    **CAUTION** This function assumes that `D_invs` is complete with 0s along        the diagonal.
  * `distance_algorithm`: distance489503algorithm to use in computing distances. Called   if `D_invs` is not specified
  * `parallel_approach`: `fragmentation` will automatically try to implement    parallel calculation if `try_parallel(graph) == true`. `parallel_approach`   can take one of three values:

      * `:auto`: choose based on `try_parallel(graph)`
      * `:parallel`: Force a parallel implementation (slower on small graphs)
      * `:serial`: Force a serial implementation (slower on large graphs)
  * `use_distance_weighting`: use distance-weigthed fragmentation? If False,    defaults to adjacency only.
  * `kwargs...`: passed to distance algorithm. Include options for heap vectors    etc.


# Let's try calculating fragmentation on some graphs


In [21]:
# generate a random graph and a wheel graph
graph = Graphs.SimpleGraphs.random_regular_graph(5000, 5);
graph_wheel = Graphs.SimpleGraphs.wheel_graph(1000);


Dict{Symbol, DistributedArrays.DArray} with 6 entries:
  :heap_data         => [0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0]
  :parents           => [0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0]
  :dists             => [0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0]
  :heap_index        => [0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0]
  :size              => [0 0 … 0 0]
  :heap_index_lookup => [0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0]

# Compare serial and distributed approach
- Simple functions that iterate over each source vertex and calculate a metric


###   Serial approach requires us to specify `parallel_approach = :serial`
Run once to compile before benchmarking

In [42]:
@time GraphFragments.fragmentation(graph, parallel_approach = :serial)

  0.660518 seconds (94.54 k allocations: 408.978 MiB, 4.36% gc time)


0.8226992892864285

In [43]:
@benchmark GraphFragments.fragmentation(graph, parallel_approach = :serial)

BenchmarkTools.Trial: 9 samples with 1 evaluation per sample.
 Range (min … max):  583.475 ms … 648.969 ms  ┊ GC (min … max): 1.32% … 5.37%
 Time  (median):     617.871 ms               ┊ GC (median):    2.99%
 Time  (mean ± σ):   613.355 ms ±  22.219 ms  ┊ GC (mean ± σ):  3.17% ± 1.30%

  █  █         █   █              ██          ██              █  
  █▁▁█▁▁▁▁▁▁▁▁▁█▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁██▁▁▁▁▁▁▁▁▁▁██▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  583 ms           Histogram: frequency by time          649 ms <

 Memory estimate: 408.98 MiB, allocs estimate: 94540.

###   To take advantage of parallelization, we should be explicit about the algorithm we're using and leverage cache arrays

- It's better to be explicit and set `parallel_approach = :parallel`
- Use `spawn_arrays` to prealocate array space for a graph `graph`
    - allows for `DistributedArrays` (recommended) and `SharedArrays` (not-recommended, performs worse on most graphs)
- This applies to fragmentation calculation and use of `DiscreteGraphAlgorithms` and `IterativeHeaps`
- Call `spawn_arrays` to build a dictionary of pre-allocated, shared (accessible by different processes) arrays
    - Default are `DistributedArrays`
- Compile before benchmarking by running once

In [44]:
algo = :dijkstra_kary

dict_arrays = spawn_arrays(graph, algo; )

@time GraphFragments.fragmentation(
    graph, 
    dict_arrays; 
    distance_algorithm = algo, 
    parallel_approach = :parallel,
)

  0.270981 seconds (46.50 k allocations: 9.573 MiB)


0.8226992892864287

**NOTE:** `dict_arrays` is shared and used to store intermediate calculations, including heap variables, parents, and distances. Its contents should not be modified.

In [46]:
@benchmark GraphFragments.fragmentation(
    graph, 
    dict_arrays; 
    distance_algorithm = algo, 
    parallel_approach = :parallel,
)

BenchmarkTools.Trial: 22 samples with 1 evaluation per sample.
 Range (min … max):  231.993 ms … 243.487 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     235.568 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   236.406 ms ±   3.396 ms  ┊ GC (mean ± σ):  0.29% ± 0.75%

  ▁   ▁ ██ ▁▁ ▁▁ ▁      ▁ ▁   ▁  ▁ █▁      ▁            ▁▁    ▁  
  █▁▁▁█▁██▁██▁██▁█▁▁▁▁▁▁█▁█▁▁▁█▁▁█▁██▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁██▁▁▁▁█ ▁
  232 ms           Histogram: frequency by time          243 ms <

 Memory estimate: 9.57 MiB, allocs estimate: 46493.

# Next, let's compare fragmentation calculation on the wheel graph

- Some graphs are relatively efficient in serial
- 1000 node wheel graph is only _slightly_ faster in parallel

In [50]:
# serial
@time GraphFragments.fragmentation(
    parallel_approach = :serial
)

  0.015406 seconds (16.54 k allocations: 20.390 MiB)


0.498

In [55]:
@benchmark GraphFragments.fragmentation(
    graph_wheel, 
    parallel_approach = :serial
)

BenchmarkTools.Trial: 656 samples with 1 evaluation per sample.
 Range (min … max):  6.772 ms …  13.325 ms  ┊ GC (min … max): 0.00% … 48.20%
 Time  (median):     7.088 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.623 ms ± 930.101 μs  ┊ GC (mean ± σ):  7.73% ± 10.23%

     ▅██▁                                                      
  ▃▃█████▄▃▃▂▃▂▂▂▁▂▂▁▂▁▁▁▁▁▂▁▁▁▁▁▂▁▃▄▆▆▆▅▃▃▃▃▃▂▂▃▁▂▂▁▂▁▁▂▂▂▁▂ ▃
  6.77 ms         Histogram: frequency by time          10 ms <

 Memory estimate: 20.39 MiB, allocs estimate: 16537.

In [53]:
algo = :dijkstra_kary

dict_arrays_wheel = spawn_arrays(graph_wheel, algo; )

@time GraphFragments.fragmentation(
    graph_wheel, 
    dict_arrays_wheel; 
    distance_algorithm = algo, 
    parallel_approach = :parallel,
)

  0.060686 seconds (6.21 k allocations: 1.864 MiB)


0.498

In [54]:
@benchmark GraphFragments.fragmentation(
    graph_wheel, 
    dict_arrays_wheel; 
    distance_algorithm = algo, 
    parallel_approach = :parallel,
)

BenchmarkTools.Trial: 751 samples with 1 evaluation per sample.
 Range (min … max):  5.332 ms … 56.364 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     5.560 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.663 ms ±  5.510 ms  ┊ GC (mean ± σ):  1.35% ± 5.93%

  █▂▁▂▂                                                       
  █████▇▄▁▄▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▄▁▁▁▁▁▄▄ ▇
  5.33 ms      Histogram: log(frequency) by time       45 ms <

 Memory estimate: 1.86 MiB, allocs estimate: 6215.

# Try different algorithms